# Timeline Analysis
Scope of the following analysis is to investigate whether the previously analyzed hashtags actually play a significant role in the underlying network structure: does a specific topic (hashtag) strengthen ties or increases diversity within Twitter users? The idea is therefore to build a metric and extract this information by combining mutual interactions Tweets and the network structure that represented the sole core of the analysis so far.

In [1]:
import snap
from snap import TUNGraph
import os
import sys
import operator
import pandas as pd
import subprocess
import numpy as np
import csv
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib import dates as mdates
import seaborn as sns
from __future__ import print_function
from IPython.display import HTML, display
import tabulate
import json
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
from collections import Counter
import re
from itertools import combinations

# Set Seaborn defaults
sns.set()
%matplotlib inline
pd.set_option("display.precision", 6)
mpl.rcParams['figure.dpi'] = 100
mpl.rcParams['savefig.dpi'] = 100
mpl.rcParams['figure.autolayout'] = True

# Global variables
data_dir = "../data"
pictures_path = os.path.join("../Pictures", "8.TimelineAnalysis")
tweets_path = "../lib/GetOldTweets-python/out/completed"

## 0. Creation of Model Classes
First of all, I might need to be a bit more concerned about performance since some Tweets files are pretty big (a few GBs), therefore I'd like to optimize some operations as much as possible.
- Since Tweets come naturally with a unique ID, I may create a `Tweet` custom type;
- Class `Interaction` representing an interaction between 2 users.

In [2]:
class Tweet:
    def __init__(self, tweet_id, users, tweet_dict):
        self.tweet_id = tweet_id
        self.tweet_dict = tweet_dict
        self.users = users
        
    def __eq__(self, other):
        if isinstance(other, Tweet):
            return self.tweet_id == other.tweet_id
        return NotImplemented
    
    def __ne__(self, other):
        x = self.__eq__(other)
        if x is not NotImplemented:
            return not x
        return NotImplemented
    
    def __hash__(self):
        return hash(self.tweet_id)

In [3]:
class Interaction:        
    def __init__(self, source, target, periods=None):
        self.source = source
        self.target = target
        self.periods = periods
        
    def __eq__(self, other):
        if isinstance(other, Interaction):
            return (self.source == other.source and self.target == other.target) or (self.source == other.target and self.target == other.source)
        return NotImplemented
    
    def __ne__(self, other):
        x = self.__eq__(other)
        if x is not NotImplemented:
            return not x
        return NotImplemented
    
    def __hash__(self):
        return hash(hash(self.source)+hash(self.target))

Hereby a collection of some utility functions:

In [4]:
def get_relative_percentage(n,m):
    return n*100.0/m

def load_graph_from_backup(filename):
    FIn = snap.TFIn("../data/"+filename+".bin")
    graph = snap.TUNGraph.Load(FIn)
    return graph

def read_large_file(file_object):
    while True:
        data = file_object.readline()
        if not data:
            break
        yield data.rstrip('\n')
        
def process_edge_line(line):
    source, target, prop = line.split(',')
    return int(source), int(target), [int(p) for p in prop.split(';')]
        
def get_usernames_from_basic_tweet_info(hashtag, tweet):
    usernames = set()
    # (1): Has tweeted using hashtag
    if hashtag in [h.lower() for h in tweet["entities"]["hashtags"]]:
        usernames.add(tweet["user"]["screen_name"].lower())

    # (2): Has been mentioned / replied to
    if not tweet["in_reply_to_screen_name"] is None:
        usernames.add(tweet["in_reply_to_screen_name"].lower())
    for mentions in tweet["entities"]["user_mentions"]:
        usernames.add(mentions["screen_name"].lower())
    return usernames

def get_tweet_usernames(hashtag, tweet):
    usernames = set()
    usernames.update(get_usernames_from_basic_tweet_info(hashtag, tweet))
    if "retweeted_status" in tweet:
        usernames.update(get_usernames_from_basic_tweet_info(hashtag, tweet["retweeted_status"]))
    if "quoted_status" in tweet:
        usernames.update(get_usernames_from_basic_tweet_info(hashtag, tweet["quoted_status"]))
    return usernames

def extract_hashtag_usernames(hashtag, tweets):
    hashtag_usernames = set()
    for tweet in tweets:
        hashtag_usernames.update(tweet.users)
    print("Total unique usernames involved in '#%s' hashtag conversations from %d tweets: %d" %(hashtag, len(tweets), len(hashtag_usernames)))
    return hashtag_usernames

# Extract tweets given a specific hashtag
def get_tweets(hashtag):
    tweets_filename = os.path.join(tweets_path,"tweets_#" + hashtag + "_2013-09-01_2016-12-31.json")
    tweets = set()
    with open(tweets_filename) as fin:
        for line in read_large_file(fin):
            tweet_dict = json.loads(line)
            tweet_id = np.int64(tweet_dict["id_str"])
            tweet_users = get_tweet_usernames(hashtag, tweet_dict)
            tweets.add(Tweet(tweet_id, tweet_users, tweet_dict))
    print("Imported %d tweets from %s" %(len(tweets),tweets_filename))
    return tweets

## 1. Extract Tweets: filter usernames and tweets

In [5]:
hashtag = "brexit"
hashtag_full = "#Brexit"

Load hashtag subgraph from backup:

In [6]:
hashtag_subgraph = load_graph_from_backup("mmr_subgraph_"+hashtag)

In [7]:
%%time
tweets = get_tweets(hashtag)

Imported 3051755 tweets from ../lib/GetOldTweets-python/out/completed/tweets_#brexit_2013-09-01_2016-12-31.json
CPU times: user 2min 26s, sys: 8.96 s, total: 2min 35s
Wall time: 2min 35s


In [8]:
hashtag_usernames = extract_hashtag_usernames(hashtag, tweets)

Total unique usernames involved in '#brexit' hashtag conversations from 3051755 tweets: 790224


In [9]:
%%time
usernames_to_id_dict = {}
with open("../data/usernames.csv") as usernames_f:
    for line in read_large_file(usernames_f):
        username = line.split(',')[0]
        encoding = int(line.split(',')[1])
        # Add to dict only if username is part of the hashtag subgraph
        if hashtag_subgraph.IsNode(encoding):
            usernames_to_id_dict[username] = encoding

CPU times: user 3min 15s, sys: 672 ms, total: 3min 16s
Wall time: 3min 16s


The next step is then to filter out those Tweets whose involved users are not part of the corresponding $H$ subgraph (i.e. if none of its involved users represents a node in $H$). The keys of the `usernames_to_id_dict` dictionary correspond to all the usernames of the $H$ subgraph, so it should be sufficient to check the following criteria:
- A tweet is kept if any of its involved users is part of the $H$ subgraph:

In [10]:
tweets_filtered = filter(lambda t: any(map(lambda u: u in usernames_to_id_dict, t.users)), tweets)
print("Number of filtered tweets (with at least 1 involved user within MMR graph data): %d (%.2f%% of %d total tweets)" %(len(tweets_filtered), get_relative_percentage(len(tweets_filtered), len(tweets)), len(tweets)))

Number of filtered tweets (with at least 1 involved user within MMR graph data): 2320935 (76.05% of 3051755 total tweets)


According to the above: among 3.051.755 total tweets, **2.320.935** involve at least one user that is part of the MMR graph data.

In [31]:
def count_tot_users(tweets):
    tot_users = set()
    for t in tweets:
        tot_users.update(t.users)
    return len(tot_users)

tot_users = count_tot_users(tweets_filtered)
print("Total number of unique users involved in %d filtered tweets: %d (%.2f%% of %d total usernames)" %(len(tweets_filtered), tot_users, get_relative_percentage(tot_users, len(hashtag_usernames)), len(hashtag_usernames)))

Total number of unique users involved in 2320935 filtered tweets: 611136 (77.34% of 790224 total usernames)


**Note**: Of course the number of users involved in the filtered tweets may be higher than the actual number of nodes in $H$, since each Tweet might include users that have not been captured by the MMR data. The reason why I decided to apply this filtering step, is that the users collected by the MMR graph data hide some extra interaction properties, e.g. 2 users are related if their total number of interactions within a 3 months period is sufficiently high, so that it represents a constant interaction over time and not a random one. This way we would already remove a lot of noisy/irrelevant data and speed up the algorithms in the next steps.

Hereby a summary of the filtering steps and the statistics about the collected tweets and usernames:

| Property | Before Filtering | After Filtering | % After/Before
|---|---|---|---|
| **Total Collected Tweets** | 3.051.755 | 2.320.935 | 76,05%
| **Total Unique Usernames** | 790.224 | 611.136 | 77,34%

## 2. User Interactions Statistics
I could first show some statistics about the interactions.

### 2.1 Top Interactions
I may show which are the interactions that occurred the most throughout the years we're considering. It's convenient to create a dictionary with $K \rightarrow V$ pairs where $K$ = interaction, $V$ = count (count how many times two people have interacted with each other within the whole timeline period we're considering).

Collecting the interactions count dictionary may be done linearly with a single pass, and the steps are hereby summarized:
1. For each tweet, create a list of all possible pairs of its involved users and for each of them create an `Interaction` instance;
2. Add a new entry to the dictionary with value 1, if the interaction is not existing;
3. If the interaction is already existing, increment its value. 

In [12]:
def create_interactions_count_dict(tweets):
    interactions_count = {}
    for t in tweets:
        pairs = list(combinations(t.users, 2))
        for p in pairs:
            i = Interaction(p[0], p[1])
            if i in interactions_count:
                interactions_count[i] += 1
            else:
                interactions_count[i] = 1
    return interactions_count

In [13]:
interactions_count = create_interactions_count_dict(tweets_filtered)

Let's visualize the results conveniently through a DataFrame, by first sorting it with descending counts (I only show the top interactions e.g. whose count value are at least 100):

In [14]:
interactions_df = pd.DataFrame(data=[(el[0].source, el[0].target, el[1]) for el in interactions_count.items()], columns=["Source", "Target", "Count"])

In [15]:
interactions_df.sort_values(by="Count", ascending=False, inplace=True)
interactions_df[interactions_df["Count"]>=100]

,Source,Target,Count
1257618,strongerin,peoplesvote_hq,7019
468747,angelneptustar,borisjohnson,3397
286710,davidjo52951945,davidjobrexit,3353
1311707,leaveeuofficial,vote_leave,2132
687961,nigel_farage,ukip,2010
1389408,nigel_farage,borisjohnson,1934
715505,peoplesvote_uk,strongerinpress,1872
716971,danieljhannan,danhannanmep,1750
273549,strongerin,vote_leave,1727
1162422,number10gov,10downingstreet,1692


The above DataFrame therefore shows who are the pairs of users that interacted the most on a time basis about hashtag **#Brexit**. Please remind that because of the filtering performed in the previous section, DataFrame might include some users that are not part of the graph data.

Another perspective may be given instead to highlight which individual users have been involved in the highest number of interactions:

In [16]:
temp1 = interactions_df.groupby(['Source']).sum().reset_index().rename(columns={'Source':'User'})
temp2 = interactions_df.groupby(['Target']).sum().reset_index().rename(columns={'Target':'User'})
temp = temp1.append(temp2)
temp = temp.groupby(['User']).sum().reset_index()
temp.sort_values(by="Count",ascending=False, inplace=True)
temp[temp["Count"]>=500]

,User,Count
408985,vote_leave,46966
281582,nigel_farage,46099
94331,david_cameron,42738
53104,borisjohnson,39051
368951,strongerin,36151
223897,leaveeuofficial,30910
41234,bbcnews,30776
356390,skynews,25889
399990,ukip,21816
321230,realdonaldtrump,21668


The result above gives an idea about who have been the most active users around the hashtag topic overtime.

In [32]:
# Backup DataFrames to visualize top interactions with NetworkX / Graphistry
interactions_df[interactions_df["Count"]>=50].to_csv(os.path.join(data_dir, "graph_visualization_top_interactions/"+hashtag+"_top_interactions.csv"),index=False)
temp[temp["Count"]>=100].to_csv(os.path.join(data_dir, "graph_visualization_top_interactions/"+hashtag+"_users_frequency.csv"),index=False)

## 3. Measuring interactions consistency overtime
What I'm now interested in, and what actually represents the end goal of this whole work, is identifying a measure of consistency related to the interactions of people taking part of controversies on Twitter. According to the data I am provided with, I may identify 2 paths:
- **MMR graph data**: there's a property of the graph data that hasn't been used so far but still lies in the CSV files I got as result of all the data transformation steps that featured the first notebooks. This property would already give a built-in definition of *interaction consistency*; however, the information we may infer from it is quite limited, and the reason is that for each pair of users we only know whether they have interacted (consistently) within a 3 months period, and given the ranges of our timeline this sums up to a total of 13 periods. It's the only way, however, to compare how interaction consistency has changed overtime;
- **Tweets**: when limiting the scope to the tweets related to a specific topic (hashtag), I have no information about which other topics all involved users have been tweeting about neither the related temporal information. Therefore, I may only know when has the *first* Twitter interaction occurred between each pair of mutually interacting users and all the subsequent ones. This is something that has already been highlighted a bit with the previous section, however I may further narrow it down and measure interactions consistency overtime if I count the interactions *per month*.

### 3.1 Long-term interaction consistency comparison: Analyzing behavioral changes overtime
The idea of this analysis is to compare interaction consistency before and after the first $H$-type interaction, in order to highlight the bonding effect of a given hashtag: does it actually strengthen or weaken ties overtime? Are users more or less likely to communicate with each other after their $H$ interaction? We want to address these questions by carrying out this first timeline analysis by using metadata as edge attributes provided by the MMR graph data.

In details: we have a total of 13 periods, ranging from 2013-09 to 2016-12, each of 3 months duration. For each edge in $H$ and for each of the periods, we then have a binary variable with values 1/0, set to 1 if the interaction occurred in the respective 3 months period, 0 otherwise (simplification of True/False values). Let's say, given period with 0-based index $i$ and edge $e$: $IsInPeriod(e,i)$ is a function as defined below:

$$
IsInPeriod(e,i) =
    \begin{cases}
        1 & \text{if $e$ occurred in period $i$,}\\
        0 & \text{otherwise.}
    \end{cases}
$$

Then, the **mutual interaction consistency** of edge $e$, calculated over $P$ consecutive periods starting from period with index $k$ until period with index $j$ (with $j +1 \ge k$), is given by:

$$MIC(e,k,j) = \frac{\sum_{i=k}^{j}IsInPeriod(e,i)}{P} = \frac{\sum_{i=k}^{j}IsInPeriod(e,i)}{j-k+1}$$

Based on the values of $j$ and $k$, $MIC$ becomes meaningful if we consider its **average** over all edges. Therefore, the **average mutual interaction consistency** $\langle MIC \rangle$ may be calculated in three similar ways:
- $\langle MIC \rangle_{T}$: for each edge $e$, $P = 13$, $k=0$, $j=12$ (the whole timeline period);
- $\langle MIC \rangle_{B}$: for each edge $e$, $P$ is the number of consecutive periods *before* the first $H$ interaction between nodes of edge $e$, i.e. if the first interaction occurred in period with index $q$, then $k=0$, $j=q-1$;
- $\langle MIC \rangle_{A}$: for each edge $e$, $P$ is the number of consecutive periods *after* the first $H$ interaction between nodes of edge $e$, i.e. $k=q$, $j=12$.

In general, given $E$ as the set of edges of $H$ subgraph:

$$\langle MIC \rangle = \frac{\sum_{e \in E}MIC(e)}{|E|}$$

where the values of $k$ and $j$, that depend directly on the considered edge $e$ based on which type of $\langle MIC \rangle$ we're calculating, have been encapsulated inside the $MIC$ function.

Values of $\langle MIC \rangle_{T}$ may then be compared across different $H$ subgraphs, whereas $\langle MIC \rangle_{B}$ and $\langle MIC \rangle_{A}$ may be then directly compared within the same $H$ subgraph to quantify the bonding impact of hashtag $H$ overtime.

As a first step, we need to conveniently store the edges attribute. This metadata is already transformed into an array of integers where values are the encoding of the periods, ranging from 0 to 12, where period $0$ is period between 2013-09 and 2013-11 (edges included),  and period $12$ between 2016-09 and 2016-11.

I may reuse the `Interaction` custom type to store them as keys of a dictionary (this time I also add a value for the periods field), and I map each `Interaction` to the period corresponding to the first $H$ interaction.

In [17]:
%%time
# Read MMR encoded file
interaction_to_first_occ = {}
count = 0
tot_lines = !wc -l ../data/mmr_encoded_final.csv
tot_lines = int(tot_lines[0].split()[0])
checkpoint = tot_lines/10
with open("../data/mmr_encoded_final.csv") as fin:
    for line in read_large_file(fin):
        source, target, periods = process_edge_line(line)
        count+=1
        if count % checkpoint == 0:
            print("Processed %d lines..." %count)
        if not hashtag_subgraph.IsEdge(source,target):
            continue
        interaction_to_first_occ[Interaction(source, target, periods)] = None
print("Done!")

Processed 43419395 lines...
Processed 86838790 lines...
Processed 130258185 lines...
Processed 173677580 lines...
Processed 217096975 lines...
Processed 260516370 lines...
Processed 303935765 lines...
Processed 347355160 lines...
Processed 390774555 lines...
Processed 434193950 lines...
Done!
CPU times: user 26min 8s, sys: 5.38 s, total: 26min 13s
Wall time: 26min 19s


I also need to have a dict that maps each period with the corresponding encoding:

In [18]:
def get_periods_encoding_dict():
    periods_encoding_df = pd.read_csv("../data/period_encoding.csv")
    return {r["period"]:r["encoding"] for _,r in periods_encoding_df.iterrows()}

In [19]:
periods_encoding_dict = get_periods_encoding_dict()

Furthermore, I need a function that given a date returns the period it belongs to:

In [20]:
def get_period_encoding_by_date(date, periods_encoding_dict):
    for period,encoding in sorted(periods_encoding_dict.iteritems(), key=operator.itemgetter(1)):
        if date >= datetime.datetime.strptime(period, "%Y-%m").date():
            result = encoding
        else:
            break
    return result

Now I may scan all the tweets and save the period when each interaction first occurred:

In [21]:
%%time
for t in tweets_filtered:
    date = datetime.datetime.strptime(t.tweet_dict['created_at'],'%a %b %d %H:%M:%S +0000 %Y').date()
    period_encoding = get_period_encoding_by_date(date, periods_encoding_dict)
    potential_edges = list(combinations(t.users, 2))
    for e in potential_edges:
        if e[0] in usernames_to_id_dict and e[1] in usernames_to_id_dict:
            interaction = Interaction(usernames_to_id_dict[e[0]],usernames_to_id_dict[e[1]])
            if interaction in interaction_to_first_occ and (interaction_to_first_occ[interaction] is None or period_encoding < interaction_to_first_occ[interaction]):
                interaction_to_first_occ[interaction] = period_encoding

CPU times: user 5min, sys: 2.01 s, total: 5min 2s
Wall time: 5min 2s


In [22]:
print("Total interactions that have been captured by MMR graph data: %d" %len([k for k in interaction_to_first_occ if not interaction_to_first_occ[k] is None]))

Total interactions that have been captured by MMR graph data: 143738


In [23]:
interaction_to_first_occ = {k:interaction_to_first_occ[k] for k in interaction_to_first_occ if not interaction_to_first_occ[k] is None}

#### 3.1.1 Average Mutual Interaction Consistency (Total)

In [24]:
def calculate_avg_mic_total(interactions_dict):
    s = pd.Series([len(i.periods)*1.0/13 for i in interactions_dict], dtype=np.float32)
    avg = s.describe()["mean"]
    return avg, pd.DataFrame(s, columns=["MIC Total"])

In [25]:
avg_mic_total, mic_total_df = calculate_avg_mic_total(interaction_to_first_occ)
mic_total_df.to_csv("../data/mic_data/mic_total_"+hashtag+".csv", index=False)
print("Average Mutual Interaction Consistency (Total) for hashtag %s: %.6f" %(hashtag_full, avg_mic_total))

Average Mutual Interaction Consistency (Total) for hashtag #Brexit: 0.225377


#### 3.1.2 Average  Mutual Interaction Consistency (Before vs After)

In [26]:
def calculate_avg_mic_before_after(interactions_dict):
    mic_before_list = []
    mic_after_list = []
    for i in interactions_dict:
        first_interaction = interactions_dict[i]
        sorted_periods = sorted(i.periods)
        periods_before = len(range(0,first_interaction))
        mic_before = len([p for p in sorted_periods if p < first_interaction])*1.0/periods_before if periods_before!=0 else 0
        mic_before_list.append(mic_before)
        mic_after_list.append(len([p for p in sorted_periods if p >= first_interaction])*1.0/len(range(first_interaction,13)))
    s_before = pd.Series(mic_before_list, name="MIC Before", dtype=np.float32)
    s_after = pd.Series(mic_after_list, name="MIC After", dtype=np.float32)
    avg_mic_before = s_before.describe()["mean"]
    avg_mic_after = s_after.describe()["mean"]
    return avg_mic_before, avg_mic_after, pd.concat([s_before, s_after], axis=1)

In [27]:
avg_mic_before, avg_mic_after, mic_before_after_df = calculate_avg_mic_before_after(interaction_to_first_occ)
mic_before_after_df.to_csv("../data/mic_data/mic_before_after_"+hashtag+".csv", index=False)
print("Average Mutual Interaction Consistency (Before) for hashtag %s: %.6f" %(hashtag_full, avg_mic_before))
print("Average Mutual Interaction Consistency (After) for hashtag %s: %.6f" %(hashtag_full, avg_mic_after))

Average Mutual Interaction Consistency (Before) for hashtag #Brexit: 0.179862
Average Mutual Interaction Consistency (After) for hashtag #Brexit: 0.458755


| Property | Value
|---|---|
| $\langle MIC \rangle_{T}$ | 0.225377
| $\langle MIC \rangle_{B}$ | 0.179862
| $\langle MIC \rangle_{A}$ | 0.458755

I'd like to investigate also for how many interactions there has been an improvement in terms of bonding effect, meaning that the two users have been interacting more frequently than they had before because of the $H$ interaction:

In [28]:
mic_before_after_df["Bonding"] = mic_before_after_df["MIC After"] > mic_before_after_df["MIC Before"]

In [29]:
mic_before_after_df["Bonding"].describe()

count     143738
unique         2
top         True
freq       94344
Name: Bonding, dtype: object

### 3.2 Detailed Interaction consistency: how long do ties survive?
This last step of the timeline analysis has the goal to provide a detailed overview of the frequency of the interactions of users for a specific hashtag per month.